# Table of Contents

1. Import Libraries
2. Load Dataset
3. Preprocessing and Exploring Text Data
  
  3.1 Text Cleaning
  
  3.2 Finding Word Count

  3.3 Find and Replace Rare Words with "Unknown" Token

4. Data Preparation

  4.1 Prepare Sequences

  4.2 Create Token-Integer Mappings

  4.3 Split Data into Train and Validation Sets

5. Model Building

  5.1 Define and Train BERT Model



# 1. Import Libraries

In [ ]:
!pip install transformers

import pandas as pd
import numpy as np
import pickle
import re
import random
from tqdm import tqdm_notebook
import torch
from torch import nn
import torch.nn.functional as F

from transformers import pipeline
from transformers import BertTokenizer, BertModel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 4.2 MB/s 
     |████████████████████████████████| 6.6 MB 35.4 MB/s 
     |████████████████████████████████| 101 kB 10.8 MB/s 
     |████████████████████████████████| 596 kB 42.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
# reproducing same results
SEED = 2019

# torch
torch.manual_seed(SEED)

# 2. Load Dataset

In [ ]:
# open text file and read in data
with open("dialogs_dataset", "rb") as f:
  dialogs = pickle.load(f)

In [ ]:
# number of text sequences
len(dialogs)

64776

In [ ]:
# print 10 random dialogs
random.sample(dialogs, 10)

['then take my card to order it',
 'No, he took an early flight',
 "Let's do the two o'clock time slot",
 'Great just what we need',
 'No, thank you',
 'No, thank you assistant',
 'YES we do, please place that order, thanks',
 'I will! Have a great night',
 'Yeah, we would like to dine outside',
 'My car is shaking a lot']

# 3. Preprocessing and Exploring Text Data

## 3.1 Text Cleaning

In [ ]:
# text cleaning
dialogs_clean = []

for i in dialogs:
  # remove everything except alphabets
  i = re.sub("[^a-zA-Z' ]", "", i)
  # convert text to lowercase
  i = i.lower()
  # add cleaned text to the list
  dialogs_clean.append(i)

In [ ]:
random.sample(dialogs_clean, 10)

[' thank you very much for your assistance',
 'yes that will be fine',
 'yes thank you once again have a good day',
 'the cinemax theater',
 'no i would like to make me own',
 "i'm at penn station in new york city and i need to go to the chelsea piers",
 'hello i need a ride can you order me a car',
 'premium again please',
 'i want to see captain marvel',
 'please be on time']


## 3.2 Finding Word Count

In [ ]:
# get list of all the words
all_words = " ".join(dialogs_clean).split()

words_dict = {}

# add word-count pair to the dictionary
for word in all_words:   
  # check if the word is already in dictionary 
  if word in words_dict:
    # increment count of word by 1 
    words_dict[word] = words_dict[word] + 1
  else:
    # add the word to dictionary with count 1 
    words_dict[word] = 1

In [ ]:
# prepare a dataframe
words_df = pd.DataFrame({'word':list(words_dict.keys()), 'count':list(words_dict.values())})

# sort words by their count in increasing order
words_df = words_df.sort_values(by = ['count'])

# reset dataframe index
words_df.reset_index(inplace = True, drop=True)

In [ ]:
# vocabulary size
len(words_df)

11147

In [ ]:
words_df.head()

word  count
0  uppermiddle      1
1       shoots      1
2        geesh      1
3       andrea      1
4      precice      1

In [ ]:
words_df.tail()

word  count
11142  you  11909
11143    a  13380
11144   to  14000
11145  the  15406
11146    i  19654

## 3.3 Find and Replace Rare Words with "Unknown" Token

In [ ]:
# user specified threshold value
rare_thresh = 4

# get percentage of rare words in the vocabulary
rare_words_count = len(words_df[words_df['count'] < rare_thresh]['word'])
total_words = len(words_df) 
rare_dist = rare_words_count / total_words

# coverage percentage of rare words in the corpus
rare_cover = words_df[words_df['count'] < rare_thresh]['count'].sum()/words_df['count'].sum()

In [ ]:
print(f"Rare words distribution in the vocabulary: {rare_dist*100:.2f}")
print(f"Rare words coverage in the corpus: {rare_cover*100:.2f}")

Rare words distribution in the vocabulary: 69.03
Rare words coverage in the corpus: 2.27


In [ ]:
# extract rare words in a list
rare_words = words_df[words_df['count'] < rare_thresh]['word'].tolist()

Let's see the technique that we will use to replace the rare words/tokens in the dataset with a special token known as the unknown token ("\<unk\>")

In [ ]:
## example
# specify rare words
r_words = ["day", "book"]

# build pattern
pattern = ""
for i in r_words:
  pattern+= "{}|".format(i)

print(pattern)

day|book|


In [ ]:
# removing the last element which is "|"
pattern = pattern[:-1]
print(pattern)

day|book


In [ ]:
# replace the rare words with the <unk> token
sents = ["it has been a long day", "this book is a must read"]

for d in sents:
  text = re.sub(pattern, " <unk> ", d)
  print(text)

it has been a long  <unk> 
this  <unk>  is a must read


In [ ]:
# create a text pattern from the rare words, like "word1 | word2 | word3..."
pattern = ""
for i in rare_words:
  pattern+= " {} |".format(i)

# removing the last element which is "|"
pattern = pattern[:-1]

# empty list 
dialogs_clean_v2 = []

# replace the rare words with the <unk> token
for d in tqdm_notebook(dialogs_clean):
  text = re.sub(pattern, " <unk> ", d)
  dialogs_clean_v2.append(text)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  del sys.path[0]


  0%|          | 0/64776 [00:00<?, ?it/s]

In [ ]:
dialogs_clean_v2[520:530]

['does it serve traditional chinese dessert',
 'how much extra time to reach <unk> ',
 'ok lets reserve a table for dinner at hakkasan',
 'hello i need to get a car please',
 'holiday inn <unk> parkconv <unk> convention center drive <unk> park il',
 'bowling alley <unk> highway <unk> park il',
 'what types of cars does uber have',
 "what's the price difference",
 'ok get me the cheapest please',
 'ok then get me the next level']

# 4. Data Preparation

## 4.1 Prepare Sequences

In [ ]:
# capture length of all the sequences
text_word_count = []
for i in dialogs_clean_v2:
  text_word_count.append(len(i.split()))
        
# plot the sequence lengths
pd.Series(text_word_count).hist(bins = 30,range=(0,30))

In [ ]:
# function to create sequences of equal length
def create_seq(text, seq_len = 5):
      
  sequences = []    
  
  if len(text.split()) > seq_len:
    for i in range(seq_len, len(text.split())):
      # select sequence of tokens
      seq = text.split()[i-seq_len:i+1]
      # append sequence to the list
      sequences.append(" ".join(seq))

    return sequences

  else:
    
    return [text]

In [ ]:
# create sequences of equal length
seqs = [create_seq(i) for i in dialogs_clean_v2]

In [ ]:
seqs[:10]

[["hi i'm looking to book a",
  "i'm looking to book a table",
  'looking to book a table for',
  'to book a table for korean',
  'book a table for korean fod'],
 ['somewhere in southern nyc maybe the',
  'in southern nyc maybe the east',
  'southern nyc maybe the east village'],
 ["we don't want to sit at",
  "don't want to sit at the",
  'want to sit at the bar',
  'to sit at the bar but',
  'sit at the bar but anywhere',
  'at the bar but anywhere else',
  'the bar but anywhere else is',
  'bar but anywhere else is fine'],
 ['what times are available'],
 ["yikes we can't do those times"],
 ['let me check'],
 ["great let's book that"],
 ["no that's it just book"],
 ['hi i would like to see',
  'i would like to see if',
  'would like to see if the',
  'like to see if the movie',
  'to see if the movie what',
  'see if the movie what men',
  'if the movie what men want',
  'the movie what men want is',
  'movie what men want is playing',
  'what men want is playing here'],
 ['yes for m

In [ ]:
# merge list-of-lists into a single list
seqs = sum(seqs, [])

In [ ]:
seqs[:15]

["hi i'm looking to book a",
 "i'm looking to book a table",
 'looking to book a table for',
 'to book a table for korean',
 'book a table for korean fod',
 'somewhere in southern nyc maybe the',
 'in southern nyc maybe the east',
 'southern nyc maybe the east village',
 "we don't want to sit at",
 "don't want to sit at the",
 'want to sit at the bar',
 'to sit at the bar but',
 'sit at the bar but anywhere',
 'at the bar but anywhere else',
 'the bar but anywhere else is']

In [ ]:
# count of sequences
len(seqs)

205346

In [ ]:
# create input and target sequences (x and y)
x = []
y = []

for s in seqs:
  x.append(" ".join(s.split()[:-1]))
  y.append(" ".join(s.split()[1:]))

In [ ]:
x[0], y[0]

("hi i'm looking to book", "i'm looking to book a")

In [ ]:
x[88543], y[88543]

('to drive to several locations', 'drive to several locations do')

## 4.2 Create Token-Integer Mappings

In [ ]:
# create integer-to-token mapping
int2token = {}
cnt = 1

for w in set(" ".join(dialogs_clean_v2).split()):
  int2token[cnt] = w
  cnt+= 1

# create token-to-integer mapping
token2int = {t: i for i, t in int2token.items()}

In [ ]:
token2int["can"], int2token[1127]

(382, 'once')

## 4.3 Split Data into Train and Validation Sets

In [ ]:
# train-validation split
# input sequences
x_tr = x[:150000]
x_val = x[150000:]

# target sequences
y_tr = y[:150000]
y_val = y[150000:]

In [ ]:

print(x_tr[:10])
print(y_tr[:10])


["hi i'm looking to book", "i'm looking to book a", 'looking to book a table', 'to book a table for', 'book a table for korean', 'somewhere in southern nyc maybe', 'in southern nyc maybe the', 'southern nyc maybe the east', "we don't want to sit", "don't want to sit at"]
["i'm looking to book a", 'looking to book a table', 'to book a table for', 'book a table for korean', 'a table for korean fod', 'in southern nyc maybe the', 'southern nyc maybe the east', 'nyc maybe the east village', "don't want to sit at", 'want to sit at the']


# 5. Model Building

## 5.1 Define and Train BERT Model


In [44]:
import os
import torch
import string
from transformers import BertTokenizer, BertForMaskedLM

# load the model 
def load_model():
  try:
    # Initializing a tokenizer using uncased bert.
    bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    # Initializing a model using eval mode to use pre-trained model.
    bert_model = BertForMaskedLM.from_pretrained('bert-base-uncased').eval()
    return bert_tokenizer,bert_model
  except Exception as e:
    pass

# decode the prediction
def decode(tokenizer, pred_idx, top_clean):
  # fetching the tokens to be ignored and appending [PAD]
  ignore_tokens = string.punctuation + '[PAD]'
  # tokens array
  tokens = []
  for w in pred_idx:
    # splitting the generated token after decoding
    token = ''.join(tokenizer.decode(w).split())
    if token not in ignore_tokens:
      # appending the token into tokens array
      tokens.append(token.replace('##', ''))
  return '\n'.join(tokens[:top_clean])

# encode the prediction
def encode(tokenizer, text_sentence, add_special_tokens=True):
  text_sentence = text_sentence.replace('<mask>', tokenizer.mask_token)
  # if <mask> is the last token, appending a "." so that models do not predict punctuation.
  if tokenizer.mask_token == text_sentence.split()[-1]:
    # appending '.' at the end.
    text_sentence += ' .'
    # converts the string to encoded form with configuartions passed
    input_ids = torch.tensor([tokenizer.encode(text_sentence, add_special_tokens=add_special_tokens)])
    #fetch the masked token id
    mask_idx = torch.where(input_ids == tokenizer.mask_token_id)[1].tolist()[0]
  return input_ids, mask_idx

# Function predicting the input statement with passing number of words to predict
def get_all_predictions(text_sentence, top_clean=5):
    # ========================= BERT =================================
    # call the encode function
  input_ids, mask_idx = encode(bert_tokenizer, text_sentence)
  with torch.no_grad():
    predict = bert_model(input_ids)[0]
  # call the decode function to decode the tokens
  bert = decode(bert_tokenizer, predict[0, mask_idx, :].topk(no_of_word_prec).indices.tolist(), top_clean)
  return bert;

# Function to add mask at the end
def get_prediction(data):
  try:
    data += ' <mask>'
    res = get_all_predictions(data, top_clean=int(no_of_word_prec))
    return res
  except Exception as error:
    pass

try:
  # number of words to get after prediction
  no_of_word_prec = 3
  bert_tokenizer, bert_model  = load_model() 
  # counter for correct predictions
  accuracy_counter = 0
  # total counter 
  total_counter = 0

  for index,x in enumerate(x_tr[:30000]):
    data = x
    res = get_prediction(data)

    if y_tr[index].split()[-1] in res.split("\n"):
      # incrementing accuracy counter
      accuracy_counter = accuracy_counter + 1
      # assigning value to total counter
      total_counter = index
      # print(index,data)
      # print("Bert Prediction :",res['bert'].split("\n"))
      # print("Dataset Prediction : ", y_tr[index].split()[-1])
      #print(index, "\n")
    
  # calculating the accuracy
  accuracy = (accuracy_counter/total_counter) * 100;
  print("Accuracy:", accuracy)

except Exception as e:
  print("SOME PROBLEM OCCURED", e)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Streaming output truncated to the last 5000 lines.

16008 

16016 

16020 

16021 

16029 

16033 

16036 

16037 

16038 

16044 

16050 

16058 

16061 

16062 

16064 

16065 

16072 

16073 

16075 

16077 

16083 

16094 

16111 

16112 

16121 

16126 

16147 

16154 

16159 

16163 

16166 

16171 

16173 

16176 

16180 

16181 

16184 

16187 

16189 

16193 

16202 

16203 

16213 

16220 

16221 

16222 

16234 

16246 

16248 

16254 

16256 

16275 

16276 

16277 

16280 

16290 

16300 

16309 

16311 

16314 

16315 

16321 

16322 

16325 

16328 

16332 

16339 

16348 

16357 

16379 

16383 

16385 

16390 

16392 

16393 

16400 

16402 

16410 

16416 

16424 

16426 

16429 

16432 

16434 

16455 

16458 

16460 

16464 

16469 

16477 

16484 

16489 

16491 

16492 

16501 

16504 

16520 

16527 

16537 

16541 

16542 

16564 

16566 

16585 

16593 

16594 

16598 

16602 

16603 

16604 

16615 

16626 

16632 

16639 

16649 

16650 

16669 

16677 

1668